In [ ]:
import numpy as np
from tqdm import tqdm
import re

# Load and preprocess

In [ ]:
# Load Shakespeare.txt
# LSTM is character-based, so we want each sonnet as a single all-lowercase string
# Punctuation included, leading spaces removed (couplet not formatted)

# This was originally written on Google Colab
# Change this section as needed to load file
downloaded = drive.CreateFile({'id':'1cwaY0yRvUNgggFytJpca_1Mao85bawQN'})
downloaded.GetContentFile('filename.txt')
f = open('filename.txt')

data = []
accum = ''

for line in f:
  # Case: just a newline
  if len(line) == 1: 
    continue
  # Case: line contains only a number--beginning of new sonnet
  elif line.strip(' ').strip('\n').isnumeric():
    if len(accum) != 0:
      accum += END_OF_SONNET
      data.append(accum)
      accum = ''
  # Case: line of sonnet
  else:
    accum += line.strip(' ').lower()

data = np.array(data)
print(f'data.shape: {data.shape}')
print('')
print(f'data[0]: {data[0]}')

In [ ]:
# Create dataset of 40-character strings
# To speed up training, we use semi-redundant sequences, picking 40-character sequences starting every n characters
# X1 is array of 40-character strings
# Y1 is array containing the next character

window = 40
n = 5

X1 = []
Y1 = []
for sonnet in data:
  for i in range(0, len(sonnet)-window-1, n):
    X1.append(sonnet[i:i+window])
    Y1.append(sonnet[i+window])

print(f'len(X1): {len(X1)}\t len(Y1): {len(Y1)}')
print('')
print(f'X1[0]: {X1[0]}')
print(f'Y1[0]: {Y1[0]}')

In [ ]:
# RNN expects each character as a boolean array

# rev_mapping: int -> char
rev_mapping = set()
rev_mapping.update(''.join(data))
rev_mapping = np.array(list(rev_mapping))

# mapping: character --> boolean array
I = np.identity(len(s))
mapping = {k:I[v] for v,k in enumerate(rev_mapping)}

X = np.array([[mapping[k] for k in seq] for seq in X1])
Y = np.array([mapping[k] for k in Y1])

print(f'X.shape: {X.shape}\t Y.shape): {Y.shape}')
print('')
print(f'X[0]: {X[0]}')
print('')
print(f'Y[0]: {Y[0]}')
print('')
print(f'rev_mapping: {rev_mapping}')

# RNN

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation

In [ ]:
rnn = Sequential([
                    LSTM(200, input_shape=(window,len(mapping))),
                    Dense(len(mapping), activation='softmax')
])
rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
rnn.fit(X, Y, epochs=100)

In [ ]:
# TODO: Um, these are still a bit nonsense. Did I do something else wrong?
def generate(seed, temperature=1):
  x = seed.lower()
  x = np.array([mapping[k] for k in x]).reshape((1,len(seed),len(s)))
  z = np.log(rnn.predict(x).flatten())
  q = np.exp(z / temperature)
  q /= np.sum(q)
  return np.random.choice(rev_mapping, p=q)

def generate_sequence(seq, temperature=1, max_len=4000):
  while seq[-1] != END_OF_SONNET and len(seq) < max_len:
    seq += generate(seq[len(seq)-40:])
  return seq

for temp in [1.5, 0.75, 0.25, 0.01]:
  seq = generate_sequence('shall i compare thee to a summer\'s day?\n',
                          temperature=temp)
  print(f'{temp}: {seq}\n\n')